<a href="https://colab.research.google.com/github/deepkick/FOSS4G_Kansai/blob/master/MLP_kerasJP_20191014_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  衛星画像の多層パーセプトロンの練習

これにはいくつかのコードが必要です。将来、独自のモデルを作成する方法がわかるように、ゆっくりとステップを説明します。

このチュートリアルで説明する手順は次のとおりです。
- ライブラリのロード
- データのロード
- モデルの定義
- モデルのコンパイル
- モデルの適合
- モデルの評価

**必要なライブラリをロードする**

Googleドライブをマウントする

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
os.chdir('/content/drive/My Drive/FOSS4G_DeepLearning_HandsOn_datasets/')

In [0]:
# ignore annoying warnings
import warnings
warnings.filterwarnings('ignore')

この演習にはKerasを使用します

In [0]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense
model = Sequential()

**準備されたデータを読み込む**

In [0]:
x_train,y_train = np.load('./isprs_vaihingen/train/patches/image.npy'),np.load('./isprs_vaihingen/train/patches/label.npy')
x_test,y_test = np.load('./isprs_vaihingen/val/patches/image.npy'),np.load('./isprs_vaihingen/val/patches/label.npy')

In [0]:
print (x_train.shape)

In [0]:
from keras import backend as K
print (K.image_data_format())

In [0]:
print (int(y_train.max()))

In [0]:
num_classes = 5
channel = x_train.shape[-1]
print (channel)

ピクセル単位のパーセプトロン学習のために、画像パッチを1Dベクトルに変形します

In [0]:
x_train = x_train.reshape(x_train.shape[0]*x_train.shape[1]*x_train.shape[2],x_train.shape[3])
x_test = x_test.reshape(x_test.shape[0]*x_test.shape[1]*x_test.shape[2],x_test.shape[3])

In [0]:
print (x_train.shape)

In [0]:
y_train = y_train.reshape(y_train.shape[0]*y_train.shape[1]*y_train.shape[2])
y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1]*y_test.shape[2])

In [0]:
print (y_train.shape)

**One-hot encoding** カテゴリー機能を数値変数に転送するのに便利です 

matplotlibを使用して1つのサンプルをプロットする

In [0]:
y_tra = keras.utils.to_categorical(y_train, num_classes)
y_tes = keras.utils.to_categorical(y_test, num_classes)

In [0]:
print(y_tra.shape)

In [0]:
print(y_tra[0,])

**モデルを定義する**

In [0]:
model = Sequential()
model.add(Dense(16, input_dim=(3), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

完全に接続されたレイヤーである密なレイヤーについては既に知っています。 **アクティベーション関数**とは何か？を見てみましょう 

**活性化関数**は、人工ニューラルネットワークの非常に重要な機能です。 彼らは基本的に、ニューロンを活性化すべきかどうかを決定します。 ニューロンが受信している情報が、与えられた情報に関連しているか、無視すべきか

**ReLU**

<center> $ A(x) = max(0,x) $ <center>

In [0]:
from IPython.display import Image
Image('fig/ReLU.jpeg', width=500, height=300)

**Sigmoid**

\begin{equation*}
A(x) = \frac{1}{(1-e^{-x})}
\end{equation*}

In [0]:
Image('fig/Sigmoid.png')

**モデルのコンパイル**

モデルを定義したら、コンパイルする必要があります。 モデルのトレーニング中に言及するいくつかの選択肢があります 

- オプティマイザー: 一般的に使用されるモデルをトレーニングする際に重みを更新する特定のアルゴリズムは、**確率的勾配降下（SGD）**です。
- 損失関数：最適化はスペースワイトをナビゲートするために使用され、最適化は損失最小化のプロセスとして定義されます（損失関数の一般的な選択には、**バイナリクロスエントロピー**、**カテゴリクロスエントロピー（ソフトマックスクロスエントロイ）**および**平均二乗誤差（MSE）**があります ）
- モデルの評価（一般的な選択肢は、**Accuracy(正解率)**、**Precision(適合率)**、**Recall(再現率)**です）

In [0]:
Image('fig/backpropog.png', width=850,height=350)

In [0]:
Image('fig/Metrics.png', width=350,height=200)

In [0]:
SGD = keras.optimizers.SGD(lr=0.01)

In [0]:
catergorical = keras.losses.categorical_crossentropy

In [0]:
accuracy = ['accuracy']

In [0]:
model.compile(loss = catergorical, optimizer = SGD,
              metrics = accuracy)

**Fit Model**

モデルをコンパイルしたら、Kerasの`fit()`関数でトレーニングできます。この関数は、エポック、batch_sizeなどのいくつかのパラメーターを指定します。

In [0]:
BATCH_SIZE= 30000
EPOCHS = 1

In [0]:
history = model.fit(x_train, y_tra,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True, validation_data = (x_test, y_tes))

In [0]:
score = model.evaluate(x_test, y_tes)
print  (score[1])

トレーニングと評価中の精度と損失を示すプロットグラフ

In [0]:
# 1 pochsだけなので、線が引けない
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# 1 pochsだけなので、線が引けない
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

テストデータを使用してモデルのパフォーマンスを評価する

**より多くの隠れ層（より深いネットワーク）でKerasの単純なネットを改善すると、**
定義されたネットワークにより密度の高い層が追加されます

In [0]:
model = Sequential()
model.add(Dense(32, input_dim=(3), activation='relu'))
model.add(Dense(32, activation='relu'))
# adding three more layers
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

**コンパイル、トレーニング、モデルのパフォーマンスを検証する**

In [0]:
model.compile(loss = catergorical, optimizer = SGD,
              metrics = accuracy)

history1 = model.fit(x_train, y_tra,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data = (x_test, y_tes))

# compare and mention the accuracy improvements

In [0]:
score1 = model.evaluate(x_test, y_tes)
print  (score1[1])

**Kerasでいくつかのハイパーチューニングパラメーターを使用してさまざまなオプティマイザー機能をテストする**

**確率的勾配降下（SGD）**として知られる一般的なオプティマイザーに注目しましょう**
Gradient Descent 最適化アルゴリズムを使用して、各エポックの後に重みが増分的に更新されます（トレーニングデータセットを渡す）。

損失関数J(⋅)、二乗誤差の合計（SSE）は、次のように記述できます。:

In [0]:
Image('fig/loss_sgd1.png', width=350,height=100)

重みの更新の大きさと方向は、コスト勾配の反対方向に一歩を踏み出すことによって計算されます

In [0]:
Image('fig/learning_sgd2.png', width=250,height=100)

ここで、ηは学習率、$ \frac{\sigma J}{\sigma wj} $は偏微分です

In [0]:
Image('fig/SGD.png', width=650,height=300)

基本的に、GDの最適化は、山から谷に登りたいハイカー（体重係数）であると考えることができます。また、各ステップは、勾配の勾配（勾配）とハイカーの脚の長さ（学習率）によって決まります。 学習率（η）が小さすぎると、ハイカーの動きが遅くなることに注意してください。 ηが高すぎる場合、ハイカーは値[1]を見逃す可能性があります。 

いくつかの学習レート調整テクニックが利用できます。**RMSprop**、**Adam**、**Adadelta**などの高度な最適化テクニックは、パラメータを自動的に調整します。

In [0]:
SGD = keras.optimizers.SGD(lr=0.01, decay = 0.005)

In [0]:
model.compile(loss = catergorical, optimizer = SGD,
              metrics = accuracy)

In [0]:
history1 = model.fit(x_train, y_tra,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data = (x_test, y_tes))

他の高度なテクニックを使用してトレーニングする

In [0]:
SGD = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

In [0]:
#SGD = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)

In [0]:
model.compile(loss = catergorical, optimizer = SGD,
              metrics = accuracy)
history1 = model.fit(x_train, y_tra,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data = (x_test, y_tes))

**見えないデータのモデル予測**

In [0]:
x_pred,y_pred = np.load('./isprs_vaihingen/test/patches/image.npy'),np.load('./dataset/isprs_vaihingen/test/patches/label.npy')

In [0]:
print (x_pred.shape)

In [0]:
x_pred = x_pred.reshape(x_pred.shape[0]*x_pred.shape[1]*x_pred.shape[2],x_pred.shape[3])

In [0]:
print (x_pred.shape)

In [0]:
Predict_prob = model.predict(x_pred)

In [0]:
Predict_prob.shape

In [0]:
Predict_prob[0]

In [0]:
Predict_class = np.argmax(Predict_prob,axis=-1)

In [0]:
Predict_class.shape

In [0]:
Predict_class = Predict_class.reshape(70,256,256)

In [0]:
%matplotlib  inline
import matplotlib.pyplot as  plt
fig = plt.figure()
ax = fig.add_subplot(121)
ax.imshow(Predict_class[4], interpolation='none')
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Predict')
fig.show()

ax = fig.add_subplot(122)
ax.imshow(y_pred[4], interpolation='none')
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Label')

fig.suptitle('Scene: top_mosaic_09cm_area1')
fig.show()


**参照**

[1] Bottou, Léon. "Stochastic gradient descent tricks." Neural Networks: Tricks of the Trade. Springer Berlin Heidelberg, 2012. 421-436.